Investigating URL Scheme listing, registration, de-registration on OS X

On a fresh conda install, `pip install pyobjc`

Dump the registry to a txt file:
```bash
# Dump the registry
/System/Library/Frameworks/CoreServices.framework/Frameworks/LaunchServices.framework/Support/lsregister -dump > lsdump.txt
```

Get a list of URL Schemes from the registry dump:
```bash
# List of schemes
cat lsdump.txt | egrep "bindings:\s*(\w*:)" \
  | awk '{gsub("bindings:", ""); gsub(",", ""); $1=$1; gsub(" ","\n"); print}' \
  | sort \
  | uniq > schemes.txt
```

Now use pyobjc to get info about the URL schemes.

In [13]:
import os
from Foundation import NSBundle, NSURL
from CoreServices import LaunchServices
import subprocess

def getBundleAppsMDFind(bundle_id):
    """Uses mdfind to search for apps associated with a BundleId
    """
    cmd = ["mdfind", f"kMDItemCFBundleIdentifier == {bundle_id}",]
    res = subprocess.run(
            cmd, 
            capture_output=True, 
            encoding="utf-8").stdout.strip().split("\n")
    return list(map(str.strip, res))


def getBundleApps(bundle_id):
    """Uses the API method directly to get Application URLs for the bundleId
    """
    res = LaunchServices.LSCopyApplicationURLsForBundleIdentifier(bundle_id, None)[0]
    return list(map(NSURL.path, res))


print(getBundleAppsMDFind("com.google.Chrome"))
print(getBundleApps("com.google.Chrome"))

['/Applications/Google Chrome.app']
['/Applications/Google Chrome.app']


In [14]:
schemes = open(os.path.expanduser("~/tmp/schemes.txt")).readlines()
#just printing out the first 5 here
for scheme in schemes[0:5]:
    scheme = scheme.replace(":","").strip()
    # request all the handlers for a scheme, returns a list of bundle identifiers
    res = LaunchServices.LSCopyAllHandlersForURLScheme(scheme)
    # Get the default handler for a scheme
    def_handler = LaunchServices.LSCopyDefaultHandlerForURLScheme(scheme)
    print("---")
    print(f"{scheme}:")
    if not res is None:
        for handler in res:
            isdef = ""
            if handler == def_handler:
                isdef = "* "
            print(f"  {isdef}{handler}")
            # Also print out the app(s) associated with the provided bundleId
            apps = getBundleApps(handler)
            for app in apps:
                print(f"    {app}")
    


---
acrobat2018:
  * com.adobe.Acrobat.Pro
    /Applications/Adobe Acrobat DC/Adobe Acrobat.app
  com.adobe.Reader
    /Applications/Adobe Acrobat Reader DC.app
---
acrobat:
  * com.adobe.Reader
    /Applications/Adobe Acrobat Reader DC.app
  com.adobe.Acrobat.Pro
    /Applications/Adobe Acrobat DC/Adobe Acrobat.app
---
adbps:
  * com.adobe.Photoshop
    /Applications/Adobe Photoshop 2020/Adobe Photoshop 2020.app
---
addressbook:
  * com.apple.AddressBook
    /System/Applications/Contacts.app
---
afp:
  * com.apple.finder
    /System/Library/CoreServices/Finder.app
  com.cocoatech.PathFinder
    /Users/vieglais/Applications/Path Finder.app
